[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/VectorInstitute/rag-bootcamp/blob/refactor/uv-migration/implementations/web_search/web_search_llamaindex.ipynb)

# Web Search with LlamaIndex

This example shows how to use the Python [LlamaIndex](https://docs.llamaindex.ai/en/stable/) library to run a text-generation request on open-source LLMs and embedding models using the OpenAI SDK, then augment that request using results from Google web search.

### 📝 Requirements

To run this notebook, you will need:


- **OpenAI API key**:  
    - Sign up at [OpenAI](https://platform.openai.com/) and create an API key

## Set up the RAG workflow environment

#### Install libraries (Only in Google Colab)

In [ ]:
import os

if "COLAB_RELEASE_TAG" in os.environ:
    # This is a Google Colab environment
    # Install required dependencies
    !pip3 install faiss-cpu langchain llama-index llama-index-core llama-index-embeddings-huggingface llama-index-llms-openai-like llama-index-vector-stores-faiss # aieng-rag-utils

#### Import libraries

In [2]:
import warnings

warnings.filterwarnings("ignore")

In [ ]:
import faiss
import os

from aieng.rag.utils import get_device_name
from aieng.rag.utils.search import DocumentReader, pretty_print

from llama_index.core import Settings, VectorStoreIndex, StorageContext
from llama_index.core.llms import ChatMessage
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.llms.openai_like import OpenAILike
from llama_index.vector_stores.faiss import FaissVectorStore

#### Load OpenAI env variables

In [4]:
OPENAI_BASE_URL = os.getenv("OPENAI_BASE_URL", "https://api.openai.com/v1")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY", "YOUR_OPENAI_API_KEY")

#### Choose LLM and embedding model

In [5]:
GENERATOR_MODEL_NAME = "gpt-4.1"
EMBEDDING_MODEL_NAME = "BAAI/bge-base-en-v1.5"

## Start with a basic generation request without RAG augmentation

Let's start by asking the model a question about recent events that it doesn't know about, something that happened after it finished training. At the time I'm writing this notebook in November 2024, the model doesn't know who won the last World Series of baseball.

*Who won the 2024 World Series of baseball?*

**The correct answer is the Los Angeles Dodgers won in October 2024.**

In [6]:
query = "Who won the 2024 World Series of baseball?"

## Now send the query to the open source model using KScope

In [7]:
llm = OpenAILike(
    model=GENERATOR_MODEL_NAME,
    is_chat_model=True,
    temperature=0,
    max_tokens=None,
    api_base=OPENAI_BASE_URL,
    api_key=OPENAI_API_KEY,
)
message = [ChatMessage(role="user", content=query)]
try:
    result = llm.chat(message)
    print(f"Result: \n\n{result}")
except Exception as err:
    if "Error code: 503" in err.message:
        print(f"The model {GENERATOR_MODEL_NAME} is not ready yet.")
    else:
        raise

Result: 

assistant: As of my latest update in June 2024, the 2024 World Series has not yet been played, so there is no winner to report. The World Series typically takes place in October. If you have a specific date in mind or need information about previous winners, feel free to ask!


The model admits that it doesn't know the answer, since according to the model it's a future event.

Let's see how we can use RAG to augment our question with a Google web search and get the correct answer.

## Ingestion: Do a Google web search for the query and obtain the necessary information

Parse through all the websites returned by a Google search, break them up into smaller digestible chunks, then encode them as vector embeddings.

In [8]:
# Do a Google web search and parse the results into a big text string
document_reader = DocumentReader(web_search_query=query, search_k=5, create_nodes=True)
docs, chunks = document_reader.load()

print(f"Number of source documents: {len(docs)}")
print(f"Number of text chunks: {len(chunks)}\n")

Number of source documents: 3
Number of text chunks: 600



#### Define the embeddings model

In [9]:
device = get_device_name()

print("Setting up the embeddings model...")
embeddings = HuggingFaceEmbedding(
    model_name=EMBEDDING_MODEL_NAME,
    device=device,
    trust_remote_code=True,
)

Setting up the embeddings model...


#### Set LLM and embedding model [recommended for LlamaIndex]

In [10]:
Settings.llm = llm
Settings.embed_model = embeddings

## Retrieval: Make the document chunks available via a retriever

The retriever will identify the document chunks that most closely match our original query. (This takes about 1-2 minutes)

In [11]:
def get_embed_model_dim(embed_model):
    embed_out = embed_model.get_text_embedding("Dummy Text")
    return len(embed_out)


faiss_dim = get_embed_model_dim(embeddings)
faiss_index = faiss.IndexFlatL2(faiss_dim)

vector_store = FaissVectorStore(faiss_index=faiss_index)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

index = VectorStoreIndex(chunks, storage_context=storage_context)

In [12]:
retriever = index.as_retriever(similarity_top_k=5)

# Retrieve the most relevant context from the vector store based on the query
retrieved_docs = retriever.retrieve(query)

Let's see what results it found. Important to note, these results are in the order the retriever thought were the best matches.

In [17]:
# Print the retrieved documents
pretty_print(retrieved_docs)

Document 1:

Dodgers win World Series 2024
----------------------------------------------------------------------------------------------------
Document 2:

The 2024 World Series was the championship series of Major League Baseball's (MLB) 2024 season. The 120th edition of the World Series, it was a best-of-seven playoff between the National League (NL) champion Los Angeles Dodgers and the American League (AL) champion New York Yankees. It was the Dodgers' first World Series appearance and win since 2020, and the Yankees' first World Series appearance since 2009. The series began on October 25 and ended on October 30 with the Dodgers winning in five
----------------------------------------------------------------------------------------------------
Document 3:

2024 World Series - Wikipedia




































Jump to content







Main menu





Main menu
move to sidebar
hide



		Navigation
	


Main pageContentsCurrent eventsRandom articleAbout WikipediaContact us




## Now send the query to the RAG pipeline

In [18]:
query_engine = RetrieverQueryEngine(retriever=retriever)
result = query_engine.query(query)
print(f"Result: \n\n{result}")

Result: 

The Los Angeles Dodgers won the 2024 World Series of baseball.


The model provides the correct answer based on the information from the web.